# Extended Kalman Filter localization example
We are considering a robot with differential traction with a rotary sensor on its top. It detects beacons and obtain a measure of its distance and angle to the robot refential frame.
Whenever we observe a beacon there is an error in the estimated angle and an error in estimating the distance. We assume that the standard deviation of the distance error increases proportionally with distance.

This work is inspired on "Laboratorial Work 4 - Robot localization based on beacons" from the course "Sistemas Robóticos Autónomos" at FEUP.
More specifically we are talking about a soccer robot in a small soccer field with the beacons in the four corner. The robot uses first a rotative of a camera and then an omnidirectional camera in the second part. 

In [4]:
'LOL'

'LOL'

## 1. Extended Kalman Filter implementation in robot simulator
Content

## 2. Implementation considering only angle or distance measure
Content

## 3. Four beacons simultaneously
Content

## 4. Four beacons: only angle or distance
Content

## 5. Observations in coordinates of the robot referential frame
Content